Step 1: Use user-item interactions to construct the graph

In [ ]:
# Define User Type
with model.rule():
    t = Train()
    User.add(user_id=t.user_id)

# Define Movie Type
with model.rule():
    item = Items()
    train = Train()
    train.item_id == item.item_id
    Movie.add(item_id=train.item_id, title=item.title)

# Link Users to Movies
with model.rule():
    t = Train()
    u = User(user_id=t.user_id)
    m = Movie(item_id=t.item_id)
    u.watched.add(m)

Step 2: Compute Item-Item similarities

In [ ]:
with model.rule():
    m1 = Movie()
    m2 = Movie()
    m1 != m2
    # Compute cosine similarity between m1 and m2
    similarity_score = movie_graph.compute.cosine_similarity(m1, m2)
    # Add objects to Similarity Type
    Similarity.add(
        movie1=Movie(item_id=m1.item_id),
        movie2=Movie(item_id=m2.item_id),
        similarity_score=similarity_score,
    )

Step 3: Predicting scores for (user, movie) pairs

In [ ]:
with model.rule():
    u = User()
    m = Movie()
    # Top k similar movie detrmined by the similarity score
    n = NearestNeighbors()

    # Remove movies already watched by users from scoring
    with model.not_found():
        m == u.watched
    # Find similar movies to movie m
    n.movie.item_id == m.item_id
    # Get intersection between watched movies and similar movies to m
    n.nearest_movies == u.watched
    # Calculate the sum similarity scores for each neighbor movie
    score_sum = aggregates.sum(n.similarity_score, per=[n.movie])
    # Define Scoring type with user, movie, and score
    Scoring.add(
        user=u,
        movie=m,
        score=score_sum
    )

Step 4: Generating Top-k Recommendations

In [ ]:
with model.rule():
    s = Scoring()
    # Get top 10 movies based on the score
    score_rank = aggregates.top(10, s.score, s, per=[s.user])

    Recommendation.add(
        user=s.user,
        score_rank = alias(score_rank, "score_rank"),
        recommended_movie = s.movie,
        score = s.score
    )

Step 5: Evaluating the item-based approach

In [ ]:
# Calculate Precision per User
with model.rule():
    u = User()
    r = Recommendation(user=u)

    # Count the number of recommended items that are relevant
    u.relevant == r.recommended_movie
    # Calculate precision for users
    true_positive = aggregates.count(u.relevant.item_id, per=[u.user_id])
    user_precision = true_positive / k_recommendations

    # Add user_precision as an attribute
    u.set(user_precision=user_precision)

# Get Average Precision
with model.query() as select:
    u = User()
    u.relevant
    user_precision = u.user_precision.or_(0.0)
    total_precision = aggregates.avg(u, user_precision)
    precision = select(
        alias(total_precision, "Average Precision")
    )